# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [1]:
#sales = graphlab.SFrame('kc_house_data.gl/')
import csv 
sales = csv.reader("kc_house_data.csv")

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [1]:
import pandas
import sklearn
#print sklearn.__version__
#pip list | grep sklearn
#pip install --upgrade sklearn
from sklearn.model_selection import train_test_split

sales = pandas.read_csv('kc_house_data.csv')

example_features = ['sqft_living', 'bedrooms', 'bathrooms']
target = ['price']

X_train, X_test, y_train, y_test = train_test_split(sales, sales[target], test_size=0.2, random_state=0)



In [3]:
X_train.head(2)
type(X_train)

pandas.core.frame.DataFrame

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [4]:

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train[example_features], y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [5]:
regr.coef_

array([[   313.17055038, -56754.66651422,   6887.71910816]])

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [6]:
regr.predict(X_test[example_features])



array([[  412168.14781776],
       [ 1325384.9064653 ],
       [  410134.06399046],
       ..., 
       [  340379.26188497],
       [  201647.75756624],
       [  426345.08621481]])

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [7]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    prediction =  model.predict(data)
    # Then compute the residuals/errors
    RSS = np.mean((prediction-outcome)**2)
    # Then square and add them up

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [8]:
rss_example_train = get_residual_sum_of_squares(regr, X_test[example_features], y_test)
print rss_example_train # should be 2.7376153833e+14

price    5.996150e+10
dtype: float64


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [9]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [37]:
sales['bedrooms_squared'] = sales['bedrooms'].apply(lambda x: x**2)
sales['bed_bath_rooms']=sales['bedrooms'].multiply(sales['bathrooms'],axis='index')
type(sales)
#X_test['bedrooms_squared'] = X_test['bedrooms'].apply(lambda x: x**2)

pandas.core.frame.DataFrame

In [19]:
from math import log
sales['log_sqft_living'] = sales['sqft_living'].apply(lambda x: log(x))


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

In [20]:
X_train.columns.values

array(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
       'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15',
       'log_sqft_living', 'lat_plus_long'], dtype=object)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [21]:
sales['lat_plus_long'] = sales['lat'].add(sales['long'], axis='index')


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [40]:
X_train, X_test, y_train, y_test = train_test_split(sales, sales[target], test_size=0.2, random_state=0)

model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
print model_1_features
model_2_features = model_1_features + ['bed_bath_rooms']
print model_2_features
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']
print model_3_features

['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms']
['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms', 'bedrooms_squared', 'log_sqft_living', 'lat_plus_long']


Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [51]:
# Learn the three models: (don't forget to set validation_set = None)
regr1 = linear_model.LinearRegression()
model_1 = regr1.fit(X_train[model_1_features], y_train)
model_1.coef_

array([[  3.12942010e+02,  -5.30962691e+04,   1.47770428e+04,
          6.53983343e+05,  -3.25707336e+05]])

In [52]:
X_train[model_2_features]
regr2 = linear_model.LinearRegression()
model_2 = regr2.fit(X_train[model_2_features], y_train)
model_2.coef_

array([[  3.06819573e+02,  -1.04604718e+05,  -7.01815289e+04,
          6.50590952e+05,  -3.09965751e+05,   2.49441497e+04]])

In [53]:
# Examine/extract each model's coefficients:
regr3 = linear_model.LinearRegression()
model_3 = regr3.fit(X_train[model_3_features], y_train)

In [54]:
model_3.coef_


array([[  5.37808086e+02,   2.78047910e+03,   1.01363766e+05,
          5.30798406e+05,  -4.09655435e+05,  -1.81822552e+04,
          7.24579939e+02,  -5.71030023e+05,   1.21142971e+05]])

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [55]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
get_residual_sum_of_squares(model_1, X_train[model_1_features], y_train)

price    5.667112e+10
dtype: float64

In [56]:
get_residual_sum_of_squares(model_2, X_train[model_2_features], y_train)

price    5.614802e+10
dtype: float64

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

In [57]:
get_residual_sum_of_squares(model_3, X_train[model_3_features], y_train)

price    5.284289e+10
dtype: float64

Now compute the RSS on on TEST data for each of the three models.

In [58]:
# Compute the RSS on TESTING data for each of the three models and record the values:

get_residual_sum_of_squares(model_1, X_test[model_1_features], y_test)

price    4.938402e+10
dtype: float64

In [59]:

get_residual_sum_of_squares(model_2, X_test[model_2_features], y_test)

price    4.875747e+10
dtype: float64

In [60]:

get_residual_sum_of_squares(model_3, X_test[model_3_features], y_test)

price    4.718299e+10
dtype: float64

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.